In [2]:
from datetime  import datetime

import pandas  as pd
import numpy   as np
import seaborn as sns
import polars  as pl

import os

In [13]:
os.chdir(path = default_path)

In [14]:
# Get the default or root folder of the user's system in case that any other person needs to run this setup
while not os.getcwd().endswith('Web Scraper'):
    os.chdir('..')
    if os.getcwd().endswith('Users'):
        print("Unable to find the root directory")
        break
users_default_path = os.getcwd()

In [17]:
data_path      = os.path.join(users_default_path,'Data')
collector_path = os.path.join(users_default_path,'Collector')


In [18]:
steam_games = os.path.join(data_path,'steam_games.csv')
video_games = os.path.join(data_path,'video_games.csv')
nyc_events  = os.path.join(data_path,'nyc-permitted-event-information.csv.zip')

In [ ]:
current_path = os.getcwd()
print(current_path)
print(os.path.abspath(path = current_path))
os.chdir("..")
#if current_path.endswith('Web Scraper'):

In [ ]:
full_path = "/Users/avaldivia/Documents/GitHub/Event-Aggregator/Web Scraper/Data/games.csv"
df = pd.read_csv(filepath_or_buffer= full_path,index_col = 0)
df.head(n = 3)

In [13]:
def k_mapping(num: str) -> int:
    multiplier = 1
    l = num
    r = 0
    if 'K' in num:
        l = num[:-1]
        multiplier = 1_000
    
    if '.' in num:
        l , r = l.split('.')

    return int( (int(l) * multiplier) + (int(r) * (multiplier/10)) )

assert(k_mapping('3.9K')   == 3900.0)
assert(k_mapping('11.9K')  == 11_900.0)
assert(k_mapping('41K')    == 41000.0)
assert(k_mapping('400')    == 400.0)

In [14]:
df['Times Listed']      = df['Times Listed'].apply(func = k_mapping)
df['Number of Reviews'] = df['Number of Reviews'].apply(func = k_mapping)
df['Plays']             = df['Plays'].apply(func = k_mapping)
df['Playing']           = df['Playing'].apply(func = k_mapping)
df['Backlogs']          = df['Backlogs'].apply(func = k_mapping)
df['Wishlist']          = df['Wishlist'].apply(func = k_mapping)

In [ ]:
date_map = lambda date: datetime.strptime(date,'%b %d, %Y').strftime(format = '%Y-%m-%d')
# Test lambda function
sample_date = 'Feb 25, 2022'
date_map(sample_date)

In [16]:
# Remove all the rows that contains 'releases on TBD' as it messes with the mapping.
df = df[df['Release Date'] != 'releases on TBD']

In [17]:
df['Release Date'] = pd.to_datetime(df['Release Date'].apply(func = date_map))

In [18]:
#df['Genres'] = df['Genres'].apply(func = lambda l: eval(l))

In [ ]:
df.head(n = 2)

In [ ]:
s = {i for item in df['Genres'] for i in eval(item)}
print(len(s))
s

In [62]:
df = df[df['Team'].apply(lambda x: isinstance(x,str))]
df.shape

In [ ]:
df['Team'].value_counts()

In [ ]:
s = {i for item in df['Team'] for i in eval(item)}
print(len(s))

In [ ]:
df.head()

In [ ]:
sns.histplot(data = df['Number of Reviews'])

In [ ]:
df.info()

In [ ]:
# Apply One Hot encoding for each of the categories
for category in s:
    df[category] = df['Genres'].apply( func = lambda x: 1 if category in x else 0)

df.drop(labels=['Genres'],axis=1)

In [ ]:
df.head(n = 2)

In [ ]:
df.iloc[:,13:]

In [66]:
inventory = pd.concat([df['Title'],df.iloc[:,13:] ], axis = 1)

In [ ]:
inventory.head(n = 2)

In [ ]:
inventory.info()

In [70]:
#inventory.to_csv("/Users/avaldivia/Documents/GitHub/Event-Aggregator/Web Scraper/Data/video_games.csv")

In [ ]:
l = df['Reviews'][0]
for item in eval(l):
    print(item)
    print()